<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Ronen_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from pathlib import Path
from glob import glob
import csv
import json
import pandas as pd
import os
os.chdir('/content/drive/My Drive/IDC/NLP/Assignment1/')
def preprocess():
  # TODO
 vocabulary = set()
 pathlist = glob('*.csv')
 for path in pathlist:
    with open(path, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        line_count = 0
        for text in csv_reader['tweet_text']:
            list_text = list(text)
            vocabulary.update(list_text)
            line_count += 1
        #print(line_count)
 return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [4]:
import nltk.data
import nltk.tokenize 
from collections import Counter
from collections import defaultdict
def find_ngrams(text, n):
    counter = Counter()
    ngram = defaultdict(lambda: defaultdict(lambda: 0))

    num_tokens, tokens = split_into_tokens(text.lower())
    for token in tokens:
        # sequence  = {}
        for i,_ in enumerate(token):
            if(i>=n - 1):
                word, char = token[i - (n - 1):i], token[i]
                nchar_word = ''.join(word) + char
                counter[nchar_word] = counter.get(nchar_word, 0) + 1
                # sequence[char] = counter[nchar_word]
                ngram[word][char] = counter[nchar_word]
                #print({word:ngram[word]})
    return ngram
def token_to_unigram(token):
    token = token.strip().strip(",.!|&-_()[]<>{}/\"'").strip()

    def has_no_chars(token):
        for char in token:
            if char.isalpha():
                return False
        return True

    if len(token) == 1 or token.isdigit() or has_no_chars(token):
        return None
    return token
def split_into_tokens(text):
    tokens = []
    for token in nltk.tokenize.WhitespaceTokenizer().tokenize(text):
    #for token in tokenize(text):
        unigram = token_to_unigram(token)
        if unigram:
            tokens.append(unigram)
    return len(tokens), tokens
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  voc_size = len(vocabulary)
  if add_one:
       # Create a non empty dict
      model = defaultdict(lambda: defaultdict(lambda: 1/voc_size))
  else:
      model = defaultdict(lambda: defaultdict(lambda: 1e-08))
  with open(data_file_path, encoding='utf-8', newline='') as csv_file:
    csv_reader = pd.read_csv(csv_file, delimiter=',')
    corpus = []
    for tweet_text in csv_reader['tweet_text']:
            corpus.append(tweet_text)
    data = find_ngrams(" ".join(corpus),n)

    for word,chars in data.items():
        words_total = float(sum(chars.values()))
        for char, count in chars.items():
            if (add_one == True):
                p = float(count + 1)/(words_total + len(vocabulary))
            else:
                p = float(count)/words_total
            model[word][char] = p
  return model
n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
#print(sum(v for v in model['abc'].values()))
print(model['ab'])
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
# for j, key in enumerate(model.keys()):
#     if j < 50:
#         print("ngram:{:s} {:f}".format(key,sum(model[key].values())))  

{'@h': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f3fef2fe840>, {'a': 0.22302158273381295, 'i': 0.2517985611510791, 'y': 0.03237410071942446, 'k': 0.0035971223021582736, 'e': 0.1223021582733813, 'o': 0.18345323741007194, 'j': 0.0035971223021582736, 's': 0.01079136690647482, 'u': 0.05755395683453238, 'p': 0.014388489208633094, 'b': 0.017985611510791366, 'w': 0.007194244604316547, 'c': 0.007194244604316547, 't': 0.02158273381294964, '2': 0.0035971223021582736, 'r': 0.007194244604316547, '…': 0.0035971223021582736, 'l': 0.0035971223021582736, 'q': 0.0035971223021582736, 'd': 0.0035971223021582736, 'v': 0.0035971223021582736, 'h': 0.0035971223021582736, 'f': 0.0035971223021582736, '5': 0.0035971223021582736, 'n': 0.0035971223021582736}), 'ha': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f3fef2fe9d8>, {'n': 0.17027538364515452, 't': 0.2991381122556233, 'p': 0.04856001681732184, 'y': 0.004624763506411604, 's': 0.0544460794618457, 'v': 0.12423796

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [15]:
import numpy as np
def compute_entropy_xi(ngram,chars):
    ll = 0.0
    for key in chars.keys():
        p = ngram[key]
        ll += np.log2(p)
    return ll

def eval(n, model, data_file):
    """
    Compute the perplexity of ngrams from model
    """
    h_x = 0
    N = 0
    with open(data_file, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        corpus = []
        for tweet_text in csv_reader['tweet_text']:
                corpus.append(tweet_text)
        data = find_ngrams(" ".join(corpus),n)
        N = 0   # Total number of words
        for word,chars in data.items():
                N += len(chars)
            #if word in model:
                ngram = model[word]
                #for i, ngram in enumerate(model.items()):
                h_x += compute_entropy_xi(ngram,chars)
            #N += len(h_X)
    # print("N = {:d}, H(x)={:f}".format(N,h_x))
    return pow(2, (-1.0 / N) * h_x)

n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
test_file_name = 'fr.csv'
preplexity = eval(n,model,test_file_name)
print("Preplexity evaluation : {:f}".format(preplexity))


#   return per
# n =3
# file_name = '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'
# model_b = lm_b(n, vocabulary,file_name , False)
# print(eval_b(2, model_b, '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'))

Preplexity evaluation : 8205.964012


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [16]:
# def match(n, add_one):
#   # n - the n-gram to use for creating n-gram models
#   # add_one - use add_one smoothing or not

#   #TODO
#   file_path = '/content/drive/My Drive/IDC/NLP/Assignment1/{}.csv'
#   vocabulary = preprocess()

#   lang = ['en' ,'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
#   df = pd.DataFrame(columns=lang, index=lang)
#   for l1 in lang:
#     l1_model = lm(n, vocabulary, file_path.format(l1), add_one)
#     for l2 in lang:
#       df.at[l1, l2] = eval(n, l1_model, file_path.format(l2))

#   return df
def match(n, add_one):
#  path_name = ''
#  csvglob = path_name + '*.csv'
 pathlist = glob('*.csv')
 num = len(pathlist)
 matrix = np.empty(shape=(num,num),dtype='float')
 headers_list = []
 i = 0
 for model_path in pathlist:
    head, tail = os.path.split(model_path)
    headers_list.append(tail.rstrip('.csv'))
    j = 0
    modeli = lm(n, vocabulary,model_path, add_one)
    for data_file_path in pathlist:
        #if ( i != j ):
        pp = eval(n,modeli,data_file_path)
        matrix[i][j] = pp
        #print ("Preplexity of model {:d} with data file {:d} is {:f}".format(i,j,pp))
        j = j+1
    i = i + 1
#  print(matrix)
 dataframe = pd.DataFrame(matrix,columns=headers_list,index=headers_list)
 print("passed dataframe")
 return dataframe

print (match(2, False))

passed dataframe
              en            tl  ...            it            nl
en    104.680299  26538.926557  ...  16990.247351  16333.255895
tl  21464.598125    115.664164  ...  17269.734773  18961.270022
pt  42828.225912  53343.346084  ...  26568.136438  32974.824962
e   23476.377747  31940.517863  ...  13854.771563  22449.276739
fr  31031.640312  38824.497546  ...  16149.260713  21603.505854
in  20072.447159  26905.964946  ...  19765.569117  19022.134802
it  26894.303926  33514.271417  ...    146.523612  23724.546126
nl  17485.803063  25036.129929  ...  15962.881518    117.112049

[8 rows x 8 columns]


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [11]:
max_n = 4 
for n in range(4): 
  print("n = {:d} add_one:{:b}".format(n+1,False))
  print(match(n+1,False)) 
  print("n = {:d} add_one:{:b}".format(n+1,True))
  print(match(n+1,True))

n = 1 add_one:0
passed dataframe
              en            tl  ...            it            nl
en  1.542050e+05  2.852441e+06  ...  9.589803e+05  1.192940e+06
tl  2.148795e+06  1.296264e+05  ...  8.041246e+05  9.525883e+05
pt  3.343599e+06  4.292095e+06  ...  1.866123e+06  1.922445e+06
e   2.088482e+06  2.822890e+06  ...  7.122024e+05  1.958616e+06
fr  3.685320e+06  3.513247e+06  ...  1.374642e+06  1.604144e+06
in  2.116823e+06  2.138560e+06  ...  1.167012e+06  1.237363e+06
it  3.089004e+06  3.456340e+06  ...  1.035854e+05  1.829794e+06
nl  2.677125e+06  3.065747e+06  ...  1.325865e+06  1.204051e+05

[8 rows x 8 columns]
n = 1 add_one:1
passed dataframe
               en            tl  ...            it            nl
en  104966.663231   8194.420050  ...  11119.150053  11252.494848
tl    7506.422166  87756.237526  ...   9683.741576   8854.887656
pt    5770.323611   4573.418358  ...   5506.509606   5112.048425
e     8267.945294   6977.303666  ...  10745.944484   6202.616023
fr    7095.

# **Good luck!**
max_n = 4
for n = 1 in range(max_n):
  data = match(n,False)
  print(data)
  data = match(n,True)
  print(data)
  